In [24]:
import pandas as pd
import numpy as np
import vectorbt as vbt

In [25]:
tlt = vbt.YFData.download(
    "ITUB3.SA", 
    start="2022-01-01"
).get("Close").to_frame()
close = tlt.Close

In [26]:
# código cria séries do pandas com o mesmo formato da série com preços de fechamento. 
short_entries = pd.DataFrame.vbt.signals.empty_like(close)
short_exits = pd.DataFrame.vbt.signals.empty_like(close)
long_entries = pd.DataFrame.vbt.signals.empty_like(close)
long_exits = pd.DataFrame.vbt.signals.empty_like(close)

In [27]:
short_entries

Date
2022-01-03 03:00:00+00:00    False
2022-01-04 03:00:00+00:00    False
2022-01-05 03:00:00+00:00    False
2022-01-06 03:00:00+00:00    False
2022-01-07 03:00:00+00:00    False
                             ...  
2024-09-02 03:00:00+00:00    False
2024-09-03 03:00:00+00:00    False
2024-09-04 03:00:00+00:00    False
2024-09-05 03:00:00+00:00    False
2024-09-06 03:00:00+00:00    False
Name: Close, Length: 672, dtype: bool

In [28]:
'''
geramos os sinais de negociação com base no dia do mês.
'''

# entra vendido no priemiro dia do mes
short_entry_mask = ~tlt.index.tz_convert(None).to_period("M").duplicated()
short_entries.iloc[short_entry_mask] = True

short_entries

Date
2022-01-03 03:00:00+00:00     True
2022-01-04 03:00:00+00:00    False
2022-01-05 03:00:00+00:00    False
2022-01-06 03:00:00+00:00    False
2022-01-07 03:00:00+00:00    False
                             ...  
2024-09-02 03:00:00+00:00     True
2024-09-03 03:00:00+00:00    False
2024-09-04 03:00:00+00:00    False
2024-09-05 03:00:00+00:00    False
2024-09-06 03:00:00+00:00    False
Name: Close, Length: 672, dtype: bool

In [30]:
# Sai 5 dias depois
short_exit_mask = short_entries.shift(5).fillna(False)
short_exits.iloc[short_exit_mask] = True

# entra comprado 7 dias antes do fim do mes
long_entry_mask = short_entries.shift(-7).fillna(False)
long_entries.iloc[long_entry_mask] = True

# sai no ultimo dia do mes
long_exit_mask = short_entries.shift(-1).fillna(False)
long_exits.iloc[long_exit_mask] = True

long_exits

Date
2022-01-03 03:00:00+00:00    False
2022-01-04 03:00:00+00:00    False
2022-01-05 03:00:00+00:00    False
2022-01-06 03:00:00+00:00    False
2022-01-07 03:00:00+00:00    False
                             ...  
2024-09-02 03:00:00+00:00    False
2024-09-03 03:00:00+00:00    False
2024-09-04 03:00:00+00:00    False
2024-09-05 03:00:00+00:00    False
2024-09-06 03:00:00+00:00    False
Name: Close, Length: 672, dtype: bool

In [15]:
pf =  vbt.Portfolio.from_signals(
    close=close,
    entries=long_entries, 
    exits=long_exits,
    short_entries=short_entries,
    short_exits=short_exits,  
    freq="1d"
)

pf.stats()

Start                         2022-01-03 03:00:00+00:00
End                           2024-09-06 03:00:00+00:00
Period                                672 days 00:00:00
Start Value                                       100.0
End Value                                     59.384188
Total Return [%]                             -40.615812
Benchmark Return [%]                          95.899424
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              43.906719
Max Drawdown Duration                 605 days 00:00:00
Total Trades                                         65
Total Closed Trades                                  64
Total Open Trades                                     1
Open Trade PnL                                 -1.34223
Win Rate [%]                                    39.0625
Best Trade [%]                                   7.8087
Worst Trade [%]                              -10

In [16]:
pf =  vbt.Portfolio.from_signals(
    close=close,
    entries=short_entries,
    exits=short_exits,
    short_entries=long_entries,
    short_exits=long_exits,
    freq="1d"
)

pf.stats()

Start                         2022-01-03 03:00:00+00:00
End                           2024-09-06 03:00:00+00:00
Period                                672 days 00:00:00
Start Value                                       100.0
End Value                                     156.62476
Total Return [%]                               56.62476
Benchmark Return [%]                          95.899424
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              16.274071
Max Drawdown Duration                 141 days 00:00:00
Total Trades                                         65
Total Closed Trades                                  64
Total Open Trades                                     1
Open Trade PnL                                 3.386999
Win Rate [%]                                     59.375
Best Trade [%]                                10.357124
Worst Trade [%]                                 